[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД


In [4]:
import requests
import os
import csv
import pandas as pd
from tqdm.auto import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse

import warnings
warnings.filterwarnings('ignore')

## Список объектов

In [5]:
# скачиваем информацию о всех строящихся объектах на сайте "наш.дом.рф" в pandas dataframe

offset = 0
df = pd.DataFrame()

while True:
        
        url = 'https://наш.дом.рф/сервисы/api/kn/object'
        params = (
                  ('offset', offset),
                  ('limit', 100),
                  ('sortField', 'devId.devShortCleanNm'),
                  ('sortType', 'asc'),
                  ('objStatus', '0')
                 )
         
        response = requests.get(url, params = params)
        reestr = response.json()
        obj = reestr.get('data').get('list')
         
        if not obj:
                   break
     
        df1 = pd.DataFrame(obj)
        df1['developer'] = df1['developer'].apply(lambda x: x['shortName'])
        df = df.append(df1, ignore_index=True)
        
        offset += 100

In [30]:
df

,objId,developer,rpdRegionCd,objAddr,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,latitude,...,objGuarantyEscrowFlg,residentialBuildings,rpdNum,pdId,buildType,objStatus,objFloorMin,objFloorMax,objCommercNm,complexShortName
0,29679,212-2,61,г Таганрог,18,2020-03-31,1519.00,0,0,47.2238,...,0,1,61-000818,124647,Жилое,0,NaN,NaN,NaN,NaN
1,13383,2МЕН ГРУПП,72,г Тюмень,68,2022-12-31,6293.78,1,0,57.1519,...,0,1,72-000095,28105,Жилое,0,4.0,20.0,NaN,NaN
2,13397,2МЕН ГРУПП,72,"г Тюмень, ул Профсоюзная, д. 56",65,2024-12-31,6248.51,0,0,57.1519,...,0,1,72-000097,28141,Жилое,0,6.0,12.0,NaN,NaN
3,13399,2МЕН ГРУПП,72,"г Тюмень, ул Профсоюзная, д. 56",0,2024-12-31,0.00,0,0,57.1519,...,0,0,72-000097,28141,Нежилое,0,1.0,1.0,NaN,NaN
4,13398,2МЕН ГРУПП,72,"г Тюмень, ул Профсоюзная, д. 56",43,2024-12-31,4812.98,0,0,57.1519,...,0,1,72-000097,28141,Жилое,0,7.0,10.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10587,28072,ЯРКИЙ МИР,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",312,2024-06-30,14788.80,1,0,55.6591,...,0,1,50-001971,96379,Жилое,0,2.0,26.0,"Жилой комплекс""Томилинский бульвар""",NaN
10588,28071,ЯРКИЙ МИР,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",288,2024-06-30,14870.40,1,0,55.6603,...,0,1,50-002025,99284,Жилое,0,26.0,26.0,"Жилой комплекс""Томилинский бульвар""",NaN
10589,28073,ЯРКИЙ МИР,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",312,2024-06-30,14872.80,1,0,55.6611,...,0,1,50-001972,96446,Жилое,0,26.0,26.0,"Жилой комплекс""Томилинский бульвар""",NaN
10590,44447,ЯРОСЛАВЛЬЗАКАЗЧИК,76,"г Ярославль, Фрунзенский район, территория в р...",127,2022-12-31,6764.22,0,0,57.5679,...,1,1,76-000546,475093,Жилое,0,6.0,6.0,NaN,NaN


In [6]:
# Cохраняем pandas dataframe в excel
df.to_csv('nash-dom-rf.csv', index = False)
# Cохраняем pandas dataframe в excel
df.to_excel('nash-dom-rf.xlsx')
# Cохраняем pandas dataframe pickle
df.to_pickle('nash-dom-rf.pkl')

In [33]:
# Cохраняем pandas dataframe в БД
import sqlite3
conn = sqlite3.connect('BDnashdom.db')  
c = conn.cursor()
df.to_sql('nash-dom', conn)

10592

# Данные по определенному объекту

In [12]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/29679'

In [32]:
# скачиваем информацию о всех строящихся объектах на сайте "наш.дом.рф" в pandas dataframe
df2 = pd.DataFrame()
for id in tqdm(df.objId):

    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    res = requests.get(url)
    r = res.json()
    obj = r.get('data')
    
    df1 = pd.json_normalize(obj)
    df2 = df2.append(df1, ignore_index=True)

  0%|          | 0/10592 [00:00<?, ?it/s]

In [34]:
df2

,id,pdId,region,address,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,objLkFinishTypeDesc,objLkFreePlanDesc,objElemParkingCnt,...,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors
0,29679,124647,61,г Таганрог,18,2020-03-31,Другое,Под ключ,Есть,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13383,28105,72,г Тюмень,68,2022-12-31,Другое,По желанию заказчика,Есть,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13397,28141,72,"г Тюмень, ул Профсоюзная, д. 56",65,2024-12-31,Другое,Без отделки,Нет,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13399,28141,72,"г Тюмень, ул Профсоюзная, д. 56",0,2024-12-31,Другое,Без отделки,Нет,125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13398,28141,72,"г Тюмень, ул Профсоюзная, д. 56",43,2024-12-31,Другое,Без отделки,Нет,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10587,28072,96379,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",312,2024-06-30,Другое,Под ключ,Нет,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10588,28071,99284,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",288,2024-06-30,Другое,Под ключ,Нет,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10589,28073,96446,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",312,2024-06-30,Другое,Под ключ,Нет,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10590,44447,475093,76,"г Ярославль, Фрунзенский район, территория в р...",127,2022-12-31,Кирпич,Другое,Нет,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Cохраняем pandas dataframe в excel
df2.to_csv('nash-dom-rf1.csv', index = False)
# Cохраняем pandas dataframe в excel
df2.to_excel('nash-dom-rf1.xlsx')
# Cохраняем pandas dataframe pickle
df2.to_pickle('nash-dom-rf1.pkl')

In [36]:
# Cохраняем pandas dataframe в БД
import sqlite3
conn1 = sqlite3.connect('BD.db')  
c1 = conn1.cursor()
df.to_sql('nash-dom2', conn1)

10592

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"

In [37]:
# скачиваем фотографии всех строящихся объектов с сайта "наш.дом.рф"
# источник: https://waksoft.susu.ru/2021/11/03/kak-ispolzuya-python-zagruzit-vse-izobrazheniya-s-veb-straniczy/

def is_valid(url):
    """
    Проверяем, является ли url действительным URL
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_images(url):
    """
    Возвращает все URL‑адреса изображений по одному `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Получено изображение"):
        img_url = img.attrs.get("src")
        if not img_url:
            # если img не содержит атрибута src, просто пропускаем
            continue
        # сделаем URL абсолютным, присоединив имя домена к только что извлеченному URL
        img_url = urljoin(url, img_url)
        # удалим URL‑адреса типа '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # наконец, если URL действителен
        if is_valid(img_url):
            urls.append(img_url)
    return urls

def download(url, pathname):
    """
    Загружает файл по URL‑адресу и помещает его в папку `pathname`
    """
    # если путь не существует, создать dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # загружаем тело ответа по частям, а не сразу
    response = requests.get(url, stream=True)
    # получить общий размер файла
    file_size = int(response.headers.get("Content-Length", 0))
    # получаем имя файла
    filename = os.path.join(pathname, url.split("/")[-1])
    # индикатор выполнения, изменение единицы измерения на байты вместо итераций (по умолчанию tqdm)
    progress = tqdm(response.iter_content(1024), f"Загружен {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # записываем прочитанные данные в файл
            f.write(data)
            # обновление индикатора выполнения вручную
            progress.update(len(data))

def main(url, path):
    # получить все изображения
    imgs = get_all_images(url)
    for img in imgs:
        # скачать для каждого img
        download(img, path)

In [38]:
# скачаем первые 10 фото объектов
for id in tqdm(df.objId[0:5]):
    url = f'https://наш.дом.рф/сервисы/каталог-новостроек/объект/{id}'
    main(url, "nash-dom")

  0%|          | 0/5 [00:00<?, ?it/s]

Получено изображение:   0%|          | 0/28 [00:00<?, ?it/s]

Загружен nash-dom\47282070:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Получено изображение:   0%|          | 0/68 [00:00<?, ?it/s]

Загружен nash-dom\47282070:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

...

Загружен nash-dom\821f36ae-fd36-4bc0-aab2-a78d9f8597bf:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

Загружен nash-dom\d7c07b91-483d-41f8-b848-afce02e1339f:   0%|          | 0.00/3.25M [00:00<?, ?B/s]

Загружен nash-dom\048ac553-8274-4ced-9731-6099b439c41f:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Загружен nash-dom\aa13d439-1dcf-413e-87c2-955e80db308a:   0%|          | 0.00/257k [00:00<?, ?B/s]

Загружен nash-dom\576f74ff-adce-44e2-ac8b-b86bdd54e934:   0%|          | 0.00/1.61M [00:00<?, ?B/s]